In [ ]:
import datetime
import matplotlib.pyplot as plt
from operator import add
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.functions import sum
from pyspark.sql import functions as F
from pyspark.sql.types import LongType

i = 1
month = '10'
times = []

while (i < 5):
    start = datetime.datetime.now()

    spark_session = SparkSession.builder.master('spark://192.168.2.11:7077')\
                                        .appName('Test with ' + str(i) + ' Cores')\
                                        .config('spark.dynamicAllocation.enabled', False)\
                                        .config('spark.dynamicAllocation.shuffleTracking.enabled', False)\
                                        .config('spark.shuffle.service.enabled', False)\
                                        .config('spark.dynamicAllocation.executorIdleTimeout', '30s')\
                                        .config('spark.executor.cores', 1)\
                                        .config('spark.cores.max', i)\
                                        .config('spark.driver.port', 9998)\
                                        .config('spark.blockManager.port', 10005)\
                                        .config('spark.executor.memory', '1024mb')\
                                        .config('spark.driver.memory', '1024m')\
                                        .getOrCreate()
    spark_context = spark_session.sparkContext
    spark_context.setLogLevel('ERROR')

    reddit_df = spark_session.read.json('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/RC_2016-' + month)
    
    reddits_with_parties = reddit_df.withColumn('Party',
                                                F.when(F.col('body').contains("Hillary")\
                                                       | F.col("body").contains("hillary")\
                                                       | F.col("body").contains("Democrat")\
                                                       | F.col("body").contains("democrat"), 'Democrat')\
                                                 .when(F.col("body").contains("Trump")\
                                                       | F.col("body").contains("trump")\
                                                       | F.col("body").contains("Republican")\
                                                       | F.col("body").contains("republican"), 'Republican')\
                                                 .otherwise('None'))
    reddits_with_parties_datetime = reddits_with_parties.withColumn("created_utc",
                                                                    F.to_utc_timestamp(F.from_unixtime(F.col("created_utc")\
                                                                                                        .cast(LongType()), 'yyyy-MM-dd'), 'EST'))
    reddits_with_parties_by_day = reddits_with_parties_datetime.groupBy("created_utc", 'Party')\
                                                               .agg(avg("controversiality"), count("controversiality"))\
                                                               .orderBy("created_utc", 'party')\
                                                               .withColumnRenamed('avg(controversiality)', 'Average')\
                                                               .withColumnRenamed('count(controversiality)', 'Count')
    
    # Save the daily stats
    reddits_with_parties_by_day.repartition(1).write.save('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/2016_' + month + '_daily_stats_' + str(i) + '_cores', format = 'json')
    spark_session.catalog.clearCache()
    spark_context.stop()
    
    i += 1
    times.append((datetime.datetime.now() - start).total_seconds())
    
spark_session = SparkSession.builder.master('spark://192.168.2.11:7077')\
                                            .appName('Save Times')\
                                            .config('spark.dynamicAllocation.enabled', False)\
                                            .config('spark.dynamicAllocation.shuffleTracking.enabled', False)\
                                            .config('spark.shuffle.service.enabled', False)\
                                            .config('spark.dynamicAllocation.executorIdleTimeout', '30s')\
                                            .config('spark.executor.cores', 1)\
                                            .config('spark.cores.max', 4)\
                                            .config('spark.driver.port', 9998)\
                                            .config('spark.blockManager.port', 10005)\
                                            .config('spark.executor.memory', '1024mb')\
                                            .config('spark.driver.memory', '1024m')\
                                            .getOrCreate()
spark_context = spark_session.sparkContext
spark_context.setLogLevel('ERROR')

spark_session.createDataFrame([{'Core': i + 1, 'Time': times[i]} for i in range(len(times))]).repartition(1).write.save('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/2016_' + month + '_times', format = 'json')
spark_context.stop()

In [1]:
import datetime
import matplotlib.pyplot as plt
from operator import add
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.functions import sum
from pyspark.sql import functions as F
from pyspark.sql.types import LongType

i = 4
month = '10'
times = []

while (i > 0):
    start = datetime.datetime.now()

    spark_session = SparkSession.builder.master('spark://192.168.2.11:7077')\
                                        .appName('Test with ' + str(i) + ' Cores')\
                                        .config('spark.dynamicAllocation.enabled', False)\
                                        .config('spark.dynamicAllocation.shuffleTracking.enabled', False)\
                                        .config('spark.shuffle.service.enabled', False)\
                                        .config('spark.dynamicAllocation.executorIdleTimeout', '30s')\
                                        .config('spark.executor.cores', 1)\
                                        .config('spark.cores.max', i)\
                                        .config('spark.driver.port', 9998)\
                                        .config('spark.blockManager.port', 10005)\
                                        .config('spark.executor.memory', '1024mb')\
                                        .config('spark.driver.memory', '1024m')\
                                        .getOrCreate()
    spark_context = spark_session.sparkContext
    spark_context.setLogLevel('ERROR')

    reddit_df = spark_session.read.json('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/RC_2016-' + month + '_' + str(i) + '_parts')
    
    reddits_with_parties = reddit_df.withColumn('Party',
                                                F.when(F.col('body').contains("Hillary")\
                                                       | F.col("body").contains("hillary")\
                                                       | F.col("body").contains("Democrat")\
                                                       | F.col("body").contains("democrat"), 'Democrat')\
                                                 .when(F.col("body").contains("Trump")\
                                                       | F.col("body").contains("trump")\
                                                       | F.col("body").contains("Republican")\
                                                       | F.col("body").contains("republican"), 'Republican')\
                                                 .otherwise('None'))
    reddits_with_parties_datetime = reddits_with_parties.withColumn("created_utc",
                                                                    F.to_utc_timestamp(F.from_unixtime(F.col("created_utc")\
                                                                                                        .cast(LongType()), 'yyyy-MM-dd'), 'EST'))
    reddits_with_parties_by_day = reddits_with_parties_datetime.groupBy("created_utc", 'Party')\
                                                               .agg(avg("controversiality"), count("controversiality"))\
                                                               .orderBy("created_utc", 'party')\
                                                               .withColumnRenamed('avg(controversiality)', 'Average')\
                                                               .withColumnRenamed('count(controversiality)', 'Count')
    
    # Save the daily stats
    reddits_with_parties_by_day.repartition(1).write.save('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/2016_' + month + '_daily_stats_' + str(i) + '_cores_' + str(i) '_parts', format = 'json')
    spark_session.catalog.clearCache()
    spark_context.stop()
    
    i -= 1
    times.append((datetime.datetime.now() - start).total_seconds())
    
spark_session = SparkSession.builder.master('spark://192.168.2.11:7077')\
                                            .appName('Save Times')\
                                            .config('spark.dynamicAllocation.enabled', False)\
                                            .config('spark.dynamicAllocation.shuffleTracking.enabled', False)\
                                            .config('spark.shuffle.service.enabled', False)\
                                            .config('spark.dynamicAllocation.executorIdleTimeout', '30s')\
                                            .config('spark.executor.cores', 1)\
                                            .config('spark.cores.max', 4)\
                                            .config('spark.driver.port', 9998)\
                                            .config('spark.blockManager.port', 10005)\
                                            .config('spark.executor.memory', '1024mb')\
                                            .config('spark.driver.memory', '1024m')\
                                            .getOrCreate()
spark_context = spark_session.sparkContext
spark_context.setLogLevel('ERROR')

spark_session.createDataFrame([{'Core': i + 1, 'Part': i + 1, 'Time': times[i]} for i in range(len(times))]).repartition(1).write.save('hdfs://192.168.2.11:9000/user/ubuntu/redditcomments/2016_' + month + '_parts_times', format = 'json')
spark_context.stop()

SyntaxError: invalid syntax (1550778279.py, line 56)